## RAG (Retrieval Augmented Generation) Implementation

In [7]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

In [9]:
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-large"
)

vector_store  = Chroma(
    collection_name="pdf_collection",
    embedding_function=embeddings,
    persist_directory="./chroma"
)

llm = ChatOpenAI(
    model="gpt-5-nano",
    temperature=0.0
    )

In [12]:

def query(query):
    prompt_template = ChatPromptTemplate.from_messages([
    ("system", "Tu es un assistant spécialisé dans l'analyse de rapports financiers (URD)."),
    ("system", "Utilise UNIQUEMENT le contexte suivant pour répondre. Si tu ne sais pas, dis-le."),
    ("system", "Contexte : {context}"),  
    ("user", "{question}")
    ])

    context = vector_store.similarity_search(query=query, k=3)

    formatted_messages = prompt_template.invoke(
        {"context": context,
         "question": query}
    )

    response = llm.invoke(formatted_messages)
    return response.content, response.usage_metadata



query("Quelle est la répartition des fournisseurs ?")

('Répartition des fournisseurs en volume d’achats (en %):\n- Europe : 63%\n- Amérique du Nord : 20%\n- Asie : 15%\n- Autres : 2%\n\n(À titre d’information complémentaire, la répartition en nombre de fournisseurs est : Europe 73%, Amérique du Nord 13%, Asie 11%, Autres 3%.)',
 {'input_tokens': 4161,
  'output_tokens': 731,
  'total_tokens': 4892,
  'input_token_details': {'audio': 0, 'cache_read': 0},
  'output_token_details': {'audio': 0, 'reasoning': 640}})